In [2]:
from __future__ import print_function
import os
import random
import argparse
import torch
import math
import numpy as np
import wandb
from lightly.loss.ntx_ent_loss import NTXentLoss
import time
from sklearn.svm import SVC

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.transforms as transforms
from torchvision.models import resnet50, resnet18
from torch.utils.data import DataLoader

from datasets.data import ShapeNetRender, ModelNet40SVM
from models.dgcnn import DGCNN, ResNet, DGCNN_partseg
from util import IOStream, AverageMeter


/Users/emanuelerucci/opt/anaconda3/envs/mycondaenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
#COSTANTI
exp_name = 'exp' # Name of the experiment
model = 'dgcnn' # Model to use, [pointnet, dgcnn]
batch_size = 16 # Size of batch)
test_batch_size = 16 # Size of batch)
epochs = 250 # number of episode to train
start_epoch = 0  # number of episode to train
use_sgd_action = 'store_true' # Use SGD
lr = 0.001 # learning rate (default: 0.001, 0.1 if using sgd)
momentum = 0.9 # SGD momentum (default: 0.9)
no_cuda = False # enables CUDA training
seed = 1 # random seed (default: 1)'
evalu = False # evaluate the model
num_points = 2048 # num of points to use
dropout = 0.5 # dropout rate
emb_dims = 1024 # Dimension of embeddings
k = 20 # Num of nearest neighbors to use
resume_action = 'store_true'  # resume from checkpoint
model_path = '' # Pretrained model path
save_freq = 50 # save frequency
print_freq = 50 # print frequency

In [12]:
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')
if not os.path.exists('checkpoints/'+exp_name):
    os.makedirs('checkpoints/'+exp_name)
if not os.path.exists('checkpoints/'+exp_name+'/'+'models'):
    os.makedirs('checkpoints/'+exp_name+'/'+'models')

In [5]:
def train (exp_name,model,batch_size,test_batch_size,epochs,start_epoch,use_sgd_action,lr,momentum,no_cuda,seed,evalu,num_points,dropout, emb_dims,k,resume_action,model_path,save_freq,print_freq, io):
    print('ciao')
    return True

In [8]:


io = IOStream('checkpoints/' + exp_name + '/run.log')
    #io.cprint(str(args))
cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
if cuda:
    io.cprint(
        'Using GPU : ' + str(torch.cuda.current_device()) + ' from ' + str(torch.cuda.device_count()) + ' devices')
    torch.cuda.manual_seed(seed)
else:
    io.cprint('Using CPU')

if not evalu:
    train(exp_name , model , batch_size, test_batch_size, epochs, start_epoch, use_sgd_action, lr, momentum, no_cuda,seed , evalu, num_points, dropout, emb_dims, k, resume_action, model_path, save_freq, print_freq,io)
# else:
#     test(args, io)

Using CPU
ciao


In [9]:
import pytorch_lightning as pl

In [13]:
def cli_main():
    # ------------
    # args
    # ------------
    parser = argparse.ArgumentParser(description='Point Cloud Recognition')
    parser.add_argument('--exp_name', type=str, default='exp', metavar='N',help='Name of the experiment')
    parser.add_argument('--model', type=str, default='dgcnn', metavar='N',choices=['dgcnn', 'dgcnn_seg'],help='Model to use, [pointnet, dgcnn]')
    parser.add_argument('--batch_size', type=int, default=16, metavar='batch_size',help='Size of batch)')
    parser.add_argument('--test_batch_size', type=int, default=16, metavar='batch_size',help='Size of batch)')
    parser.add_argument('--epochs', type=int, default=250, metavar='N',help='number of episode to train ')
    parser.add_argument('--start_epoch', type=int, default=0, metavar='N',help='number of episode to train ')
    parser.add_argument('--use_sgd', action="store_true", help='Use SGD')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',help='learning rate (default: 0.001, 0.1 if using sgd)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',help='SGD momentum (default: 0.9)')
    parser.add_argument('--no_cuda', type=bool, default=False,help='enables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',help='random seed (default: 1)')
    parser.add_argument('--eval', type=bool,  default=False,help='evaluate the model')
    parser.add_argument('--num_points', type=int, default=2048,help='num of points to use')
    parser.add_argument('--dropout', type=float, default=0.5,help='dropout rate')
    parser.add_argument('--emb_dims', type=int, default=1024, metavar='N', help='Dimension of embeddings')
    parser.add_argument('--k', type=int, default=20, metavar='N', help='Num of nearest neighbors to use')
    parser.add_argument('--resume', action="store_true", help='resume from checkpoint')
    parser.add_argument('--model_path', type=str, default='', metavar='N', help='Pretrained model path')
    parser.add_argument('--save_freq', type=int, default=50, help='save frequency')
    parser.add_argument('--print_freq', type=int, default=50, help='print frequency')
    args = parser.parse_args()


    # ------------
    # data
    # ------------
    transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    train_loader = DataLoader(ShapeNetRender(transform, n_imgs = 2), num_workers=0, batch_size=args.batch_size, shuffle=True, drop_last=True)

In [14]:
if __name__ == '__main__':
    cli_main()

usage: ipykernel_launcher.py [-h] [--exp_name N] [--model N]
                             [--batch_size batch_size]
                             [--test_batch_size batch_size] [--epochs N]
                             [--start_epoch N] [--use_sgd] [--lr LR]
                             [--momentum M] [--no_cuda NO_CUDA] [--seed S]
                             [--eval EVAL] [--num_points NUM_POINTS]
                             [--dropout DROPOUT] [--emb_dims N] [--k N]
                             [--resume] [--model_path N]
                             [--save_freq SAVE_FREQ] [--print_freq PRINT_FREQ]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/emanuelerucci/Library/Jupyter/runtime/kernel-494e94c5-1466-4235-9e7e-a3fca30f45fb.json


SystemExit: 2

In [ ]:
import torch
from pytorch_lightning import LightningModule

class Model1(LightningModule):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(10, 10)

    def forward(self, x):
        return self.layer1(x)

class Model2(LightningModule):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(10, 10)

    def forward(self, x):
        return self.layer1(x)

class TotalModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.model1 = Model1()
        self.model2 = Model2()

    def forward(self, x):
        x = self.model1(x)
        x = self.model2(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        return optimizer
